In [1]:
# 取数据
from lib.utils.lmdb_dataset import get_dataset
from root import get_root_path
import lib.utils.sdata as sd
import lib.model.index_model as im
import importlib
import os
import numpy as np
importlib.reload(sd)
importlib.reload(im)
code = '600893'
db_path = os.path.sep.join([get_root_path(),'lmdb'])
dataframe = get_dataset(db_path,code)
sdata = sd.StockData(code, datas=dataframe)
data = sdata.combine_income(ndays=5,buy=2)

D:\PROJECT_TW\git\finance\lib\utils\sdata.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.datas['Flag'] = list(map(lambda x: 0 if x <= buy else 1, self.datas.INCOME))


# 按时间统计

In [1]:
# 统计数据
from lib.utils.lmdb_dataset import get_dataset
from root import get_root_path
import lib.utils.sdata as sd
import importlib
import os
import numpy as np
importlib.reload(sd)
code = '600893'
db_path = os.path.sep.join([get_root_path(),'lmdb'])
datas = get_dataset(db_path,code)
sdata = sd.StockData(code, datas=datas)
# 5 天内上涨 2 个点
datas = sdata.combine_income(ndays=5,buy=2)
datas['weekDay'] = datas.index.dayofweek
group = datas.groupby(['weekDay','Flag'])
week_day_array = np.array([[x[0],x[1],len(y)] for x,y in group])
week_day_up = {}
# print(week_day_array)
for iday in range(5):
    select_day_array = week_day_array[np.where(week_day_array[:,0]==iday)]
    total = np.sum(select_day_array[:,2])
    prob = select_day_array[np.where(select_day_array[:,1]==1)][:,2] / total
    week_day_up[iday] = prob[0]
print(week_day_up)



{0: 0.5272108843537415, 1: 0.5198675496688742, 2: 0.46818923327895595, 3: 0.5199335548172758, 4: 0.5443886097152428}


D:\PROJECT_TW\git\finance\lib\utils\sdata.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.datas['Flag'] = list(map(lambda x: 0 if x <= buy else 1, self.datas.INCOME))


In [2]:
# 按周日期生成训练数据
'''
数据格式：
  WD, STATUS
0 0,  BUY
1 1,  HOLD
'''
# 统计数据
from lib.utils.lmdb_dataset import get_dataset
from root import get_root_path
import lib.utils.sdata as sd
import importlib
import os
import numpy as np
importlib.reload(sd)
code = '600893'
db_path = os.path.sep.join([get_root_path(),'lmdb'])
datas = get_dataset(db_path,code)
sdata = sd.StockData(code, datas=datas)
# 5 天内上涨 2 个点
datas = sdata.combine_income(ndays=5,buy=2)
datas['weekDay'] = datas.index.dayofweek
datas['Month'] = datas.index.month
print(type(datas.index))
data_out_path = os.path.sep.join([get_root_path(),'data','train','{}.txt'.format(code)])
datas.rename(columns={'weekDay':'WD'},inplace=True) 
datas.rename(columns={'Flag':'ST'},inplace=True) 
# datas['WD'] = datas['WD'].apply(lambda x:'W'+str(x+1))
# datas['Month'] = datas['Month'].apply(lambda x: 'M' + str(x))
datas['ST'] = datas['ST'].apply(lambda x: '1' if x ==1 else '0')
# print(datas['WD'])
datas[['WD','ST','Month']].to_csv(data_out_path, sep=',', index=True)
print('over ')

<class 'pandas.core.indexes.datetimes.DatetimeIndex'>
over 


D:\PROJECT_TW\git\finance\lib\utils\sdata.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.datas['Flag'] = list(map(lambda x: 0 if x <= buy else 1, self.datas.INCOME))


# 按连续天数统计

In [52]:
import pandas as pd
import random
import lib.model.base_static as bs
from lib.utils.lmdb_dataset import get_dataset
from root import get_root_path
import lib.utils.sdata as sd
import os
import importlib
importlib.reload(bs)
code = '600893'

data_out_path = os.path.sep.join([get_root_path(),'data','train','{}_static.txt'.format(code)])
db_path = os.path.sep.join([get_root_path(),'lmdb'])
data = get_dataset(db_path,code)
sdata = sd.StockData(code, datas=data.copy())
data_static = bs.count_continue_incr(data)['count_incr']
# 5 天内上涨 2 个点
datas = sdata.combine_income(ndays=5,buy=2)
datas = pd.merge(datas, data_static, left_index=True, right_index=True)
datas.rename(columns={'count_incr':'CNTS'},inplace=True) 
datas.rename(columns={'Flag':'ST'},inplace=True) 
datas['CNTS'] = datas['CNTS'].apply(lambda x:'HI' if x >=5 else 'MI' if x >= 3 else 'LO')
datas['ST'] = datas['ST'].apply(lambda x: 'BUY' if x ==1 else 'HOLD')
datas[['ST','CNTS']].to_csv(data_out_path, sep=',', index=False)


D:\PROJECT_TW\git\finance\lib\model\base_static.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row['cont_incr'] = 0
D:\PROJECT_TW\git\finance\lib\model\base_static.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row['cont_incr'] = df['cont_incr'][idx-1] + 1


In [54]:
datas = pd.read_csv(data_out_path,sep=",")
group = datas.groupby(['ST','CNTS'])
for x,y in group:
    print(x, ":", len(y))


('BUY', 'HI') : 42
('BUY', 'LO') : 1363
('BUY', 'MI') : 144
('HOLD', 'HI') : 26
('HOLD', 'LO') : 1309
('HOLD', 'MI') : 120


# 按指标信号统计

In [3]:
data_out_path = os.path.sep.join([get_root_path(),'data','train','{}_signal.txt'.format(code)])
index_keys=['MACD','MFI','BBANDS','AROON','SAR','CCI']
data_s = sdata.combine_index_signal(data, index_keys=index_keys)
for ikey in index_keys:
    data_s[ikey] = data_s[ikey].apply(lambda x: 'BS' if x==1 else 'HS')
data_s['Flag'] = data_s['Flag'].apply(lambda x: 'BUY' if x ==1 else 'HOLD')
data_s.rename(columns={'Flag':'ST'},inplace=True) 
index_keys.append('ST')
data_s=data_s.dropna()
data_s[index_keys]
data_s[index_keys].to_csv(data_out_path, sep=',', index=False)